In [2]:
import snowflake.connector
import configparser
import requests
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
# Load Hugging Face transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# Load config
config = configparser.ConfigParser()
config.read("config.ini")
SNOWFLAKE_CONFIG = {
   "user": config["snowflake"]["user"],
   "password": config["snowflake"]["password"],
   "account": config["snowflake"]["account"],
   "warehouse": config["snowflake"]["warehouse"],
   "database": config["snowflake"]["database"],
   "schema": config["snowflake"]["schema"],
}
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
API_KEY = config["huggingface"]["api_key"]
HEADERS = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

c:\Users\Anshika.Ranjan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def create_ai_solutions_table():
   conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
   cursor = conn.cursor()
   query = """
   CREATE TABLE IF NOT EXISTS AI_SOLUTIONS (
       INCIDENT_ID STRING PRIMARY KEY,
       SUBJECT STRING,
       IMPACT_LEVEL STRING,
       AFFECTED_SERVICE STRING,
       TIMESTAMP TIMESTAMP,
       DESCRIPTION STRING,
       AI_GENERATED_SOLUTION STRING
   );
   """
   cursor.execute(query)
   conn.commit()
   cursor.close()
   conn.close()
   print("AI_SOLUTIONS table checked/created successfully.")

In [4]:
def fetch_latest_email_alert():
   conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
   cursor = conn.cursor()
   query = """
       SELECT subject, INCIDENT_ID, impact_level, affected_service, timestamp, description
       FROM cleaned_alerts
       ORDER BY timestamp DESC
       LIMIT 1;
   """
   cursor.execute(query)
   latest_alert = cursor.fetchone()
   column_names = [desc[0].lower() for desc in cursor.description]
   cursor.close()
   conn.close()
   return dict(zip(column_names, latest_alert)) if latest_alert else None

In [5]:
def generate_solution(alert_data):
   if not alert_data:
       return "No new alerts found."
   formatted_alert = f"""
   Incident ID: {alert_data['incident_id']}
   Subject: {alert_data['subject']}
   Impact Level: {alert_data['impact_level']}
   Affected Service: {alert_data['affected_service']}
   Timestamp: {alert_data['timestamp']}
   Description: {alert_data['description']}
   """
   data = {"inputs": f"Analyze this email alert and provide a solution with steps:\n\n{formatted_alert}"}
   try:
       response = requests.post(API_URL, headers=HEADERS, json=data)
       response.raise_for_status()
       result = response.json()
       if isinstance(result, list) and result:
           return result[0].get("generated_text", "Solution not found.")
       return "Solution not found."
   except Exception as e:
       print(f"Error: {e}")
       return "API request failed."

In [6]:
def store_ai_solution(alert_data, ai_solution):
   conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
   cursor = conn.cursor()
   query = """
   INSERT INTO AI_SOLUTIONS (INCIDENT_ID, SUBJECT, IMPACT_LEVEL, AFFECTED_SERVICE, TIMESTAMP, DESCRIPTION, AI_GENERATED_SOLUTION)
   VALUES (%s, %s, %s, %s, %s, %s, %s)
   """
   values = (
       alert_data['incident_id'], alert_data['subject'], alert_data['impact_level'],
       alert_data['affected_service'], alert_data['timestamp'],
       alert_data['description'], ai_solution
   )
   cursor.execute(query, values)
   conn.commit()
   cursor.close()
   conn.close()
   print("Solution stored successfully.")

In [7]:
def load_existing_embeddings():
   conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
   cursor = conn.cursor()
   query = """
       SELECT INCIDENT_ID, DESCRIPTION, AI_GENERATED_SOLUTION
       FROM AI_SOLUTIONS;
   """
   cursor.execute(query)
   rows = cursor.fetchall()
   cursor.close()
   conn.close()
   if not rows:
       return [], [], []
   incident_ids = [row[0] for row in rows]
   descriptions = [row[1] for row in rows]
   solutions = [row[2] for row in rows]
   return incident_ids, descriptions, solutions

In [8]:
def build_faiss_index(descriptions):
   embeddings = embedding_model.encode(descriptions, convert_to_numpy=True)
   dim = embeddings.shape[1]
   index = faiss.IndexFlatL2(dim)
   index.add(embeddings)
   return index

In [9]:
def check_for_similar_alert(new_description, index, descriptions, incident_ids, solutions, threshold=0.85):
   new_embedding = embedding_model.encode([new_description], convert_to_numpy=True)
   D, I = index.search(new_embedding, k=1)
   similarity = 1 / (1 + D[0][0])  # Convert L2 to similarity
   if similarity >= threshold:
       match_idx = I[0][0]
       print(f"Similar alert found! Similarity: {similarity:.2f}")
       return solutions[match_idx]
   print(f"No similar alert found. Similarity: {similarity:.2f}")
   return None

In [10]:
create_ai_solutions_table()
latest_alert = fetch_latest_email_alert()
if latest_alert:
   incident_ids, descriptions, solutions = load_existing_embeddings()
   if descriptions:
       index = build_faiss_index(descriptions)
       matched_solution = check_for_similar_alert(
           latest_alert["description"], index, descriptions, incident_ids, solutions
       )
   else:
       matched_solution = None
   if matched_solution:
       store_ai_solution(latest_alert, matched_solution)
   else:
       generated_solution = generate_solution(latest_alert)
       if generated_solution:
           store_ai_solution(latest_alert, generated_solution)

AI_SOLUTIONS table checked/created successfully.
No similar alert found. Similarity: 0.41
Solution stored successfully.
